# Organização de scripts em pipelines  

## Arquitetura Geral de Diretórios

```
├── config/
│   └── spark_config.py          # Inicialização da SparkSession
├── src/
│   ├── extract/
│   │   └── load_data.py         # Lê dados de fontes (Parquet, Hive, etc.)
│   ├── transform/
│   │   ├── feature_engineering.py
│   │   └── aggregations.py
│   ├── validate/
│   │   └── quality_checks.py    # Regras de qualidade, schemas
│   ├── model/
│   │   └── train_model.py       # (se for um pipeline de ML)
│   └── utils/
│       └── helpers.py           # Funções auxiliares
├── jobs/
│   └── run_pipeline.py          # Pipeline orquestrado
├── tests/
│   └── test_transform.py
├── requirements.txt
└── README.md
```

## Modularize por fase

- Cada etapa da pipeline deve ser uma função bem definida, e cada módulo faz apenas uma coisa

```python
def add_flags(df):
    return df.withColumn("flag_novo", F.col("dias_atraso") > 90)

def enrich_with_cep(df, df_cep):
    return df.join(broadcast(df_cep), on="cep", how="left")
```

## Script principal (jobs/run_pipeline.py)

```python
from config.spark_config import get_spark
from src.extract.load_data import load_clientes, load_pagamentos
from src.transform.feature_engineering import add_flags
from src.validate.quality_checks import assert_no_nulls

spark = get_spark()

df_cli = load_clientes(spark)
df_pag = load_pagamentos(spark)

df_merged = df_cli.join(df_pag, "id_cliente")
df_final = add_flags(df_merged)

assert_no_nulls(df_final, "flag_novo")

df_final.write.mode("overwrite").parquet(".../output/df_final.parquet")
``` 

## Evite Anti-padrões

- Não coloque tudo em um único notebook
- Não acople leitura, transformação e escrita na mesma função
- Não misture lógica de negócios com parsing de argumentos ou paths

## Inclua testes simples

Com pytest, teste sua lógica local com DataFrames pequenos (via spark.createDataFrame(...)).
Isso é essencial para validação de features em modelos de risco regulatórios.